In [28]:
from typing import Union
import torch
import numpy as np
import scipy
import finufft

from numpy.random import standard_normal


class finufft1D1(torch.autograd.Function):
    """
    FINUFFT 1D problem type 1 (non-uniform points)
    """

    @staticmethod
    def forward(
        points: torch.Tensor,
        values: torch.Tensor,
        output_shape: Union[int, None] = None,
        out: Union[torch.Tensor, None] = None,
        fftshift=False,
        **finufftkwargs,
    ) -> torch.Tensor:
        n_modes = output_shape if output_shape is None else len(values)

        # TODO -- these are probably being "double processed". Should be some obvious way around this
        isign = finufftkwargs.get("isign") if "isign" in finufftkwargs else -1
        # TODO -- isign should be left alone in the case we do not fftshift
        modeord = 0 if fftshift else 1

        # TODO -- probably dimension and shape match checks?
        # finufftkwags should take in also eps, isign, ...

        # TODO -- Type checks -- which need to be complex vs. not, etc.

        nufft_out = finufft.nufft1d1(
            points.numpy(),
            values.numpy(),
            n_modes,
            isign=isign,
            modeord=modeord,
            **finufftkwargs,
        )

        return torch.from_numpy(nufft_out)


class finufft1D2(torch.autograd.Function):
    """
    FINUFFT 1d Problem type 2
    """

    @staticmethod
    def forward(
        points: torch.Tensor,
        targets: torch.Tensor,
        out: Union[torch.Tensor, None] = None,
        fftshift: bool = False,
        **finufftkwargs,
    ):
        """
        Evaluates Type 2 NUFFT on inputs
        """

        if out is not None:
            raise ValueError("In-place not implemented")

        isign = finufftkwargs.get("isign") if "isign" in finufftkwargs else -1
        modeord = 0 if fftshift else 1

        # TODO -- size checks and so on for the tensors; finufft will handle the rest of these

        nufft_out = finufft.nufft1d2(
            points.numpy(),
            targets.numpy(),
            isign=isign,
            modeord=modeord,
            **finufftkwargs,
        )

        return torch.from_numpy(nufft_out)

In [34]:
Ns = [10, 100, 1000]
c1 = standard_normal(size=Ns[0]) + 1j * standard_normal(size=Ns[0])
c2 = standard_normal(size=Ns[1]) + 1j * standard_normal(size=Ns[1])
c3 = standard_normal(size=Ns[2]) + 1j * standard_normal(size=Ns[2])

In [50]:
c = c3
ctens = torch.from_numpy(c)
N = len(c)

against_torch = torch.fft.fft(ctens)
against_scipy = torch.from_numpy(scipy.fft.fft(c))
against_numpy = torch.from_numpy(np.fft.fft(c))

finufft_out = finufft1D1.forward(
    torch.from_numpy(2 * np.pi * np.arange(0, 1, 1 / N)), ctens, N
)

In [52]:
print(torch.linalg.norm(finufft_out - against_torch) / N)
print(torch.linalg.norm(finufft_out - against_scipy) / N)
print(torch.linalg.norm(finufft_out - against_numpy) / N)

tensor(9.7285e-07, dtype=torch.float64)
tensor(9.7285e-07, dtype=torch.float64)
tensor(9.7285e-07, dtype=torch.float64)


In [47]:
c1

array([ 1.68800015+1.01064153j,  0.61552917+1.25111333j,
       -0.43091698+0.411667j  , -1.6474588 +0.20897196j,
        1.04184959-0.39853988j, -1.04149243-1.09231111j,
       -1.22796487+0.16110404j, -1.07036354+0.05301891j,
        1.48115337-0.95142228j, -1.77148772+0.15926136j])

In [55]:
points = 2 * np.pi * np.arange(0, 1, 1 / 10)
f = standard_normal(10) + 1j * standard_normal(10)

In [56]:
finufft1D2.forward(points, targets)

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

In [11]:
points.numpy()

array([0.        , 0.62831855, 1.2566371 , 1.8849556 , 2.5132742 ,
       3.1415927 , 3.7699113 , 4.3982296 , 5.0265484 , 5.6548667 ],
      dtype=float32)

In [17]:
from numpy.random import standard_normal


torch.from_numpy(standard_normal(size=10) + 1j * standard_normal(size=10))

tensor([-0.7384+0.7613j, -0.4587-0.4698j, -0.0498+1.6281j,  1.0775+0.6824j,
         0.4203-0.2157j,  0.1409-0.8150j,  0.2371-1.0232j,  2.0754+1.3288j,
         0.1662-0.0154j, -1.5926-1.7762j], dtype=torch.complex128)

In [18]:
X = torch.from_numpy(standard_normal(size=10) + 1j * standard_normal(size=10))

In [19]:
X.numpy()

array([-2.18277392+0.56268495j,  1.44170011+1.55528464j,
       -0.15480382-1.27402313j,  1.33108946-0.05318239j,
        0.66729336+0.41691941j,  1.79139712+0.06417306j,
       -0.19469785+0.11891478j,  0.70620269+0.35065945j,
       -0.28880402-0.70935561j,  0.61179117-0.1736012j ])

In [57]:
X.dtype

torch.complex128

In [58]:
Y = 2 * np.pi * torch.arange(0, 1, 1 / N)

In [59]:
Y.dtype

torch.float32

In [60]:
import finufft
import scipy

In [81]:
f_k = scipy.fft.fft(np.array([1.0, 2.0, 1.0, -1.0, 1.5]))

X = scipy.fft.ifft(f_k)

In [82]:
Y = (
    finufft.nufft1d2(
        2 * np.pi * np.arange(0, 1, 1 / 5), f_k, isign=1, modeord=1
    )
    / 5
)

In [85]:
np.linalg.norm((X - Y) / 2)

5.087739964477703e-07

In [86]:
f_k = torch.from_numpy(f_k)

In [88]:
torch.fft.ifft(f_k)

tensor([ 1.0000+0.j,  2.0000+0.j,  1.0000+0.j, -1.0000+0.j,  1.5000+0.j],
       dtype=torch.complex128)

In [1]:
from numpy.random import standard_normal
import torch
import scipy
import numpy as np

import pytorch_finufft


targets = standard_normal(10) + 1j * standard_normal(10)
N = len(targets)

inv_targets = scipy.fft.fft(targets)

inv_target_tens = torch.from_numpy(inv_targets)

In [3]:
against_torch = torch.fft.ifft(inv_target_tens)
against_scipy = scipy.fft.ifft(inv_targets)
against_numpy = np.fft.ifft(inv_targets)

In [4]:
torch.norm(against_torch - against_numpy)

tensor(8.1632e-16, dtype=torch.float64)

In [8]:
finufft_out = (
    pytorch_finufft.functional.finufft1D2.forward(
        2 * np.pi * torch.arange(0, 1, 1 / N, dtype=torch.float64),
        inv_target_tens,
        isign=1,
        # modeord=0,
    )
    / N
)

In [9]:
finufft_out

tensor([-1.4906+0.4040j, -0.5384-0.0209j,  0.0909+1.0173j, -0.3309+0.4834j,
        -0.3796-1.3598j, -1.1410+0.7581j,  2.0789-1.5162j, -0.5641-0.7436j,
        -0.2081+1.0429j, -0.3351+0.1575j], dtype=torch.complex128)

In [10]:
against_torch

tensor([-1.4906+0.4040j, -0.5384-0.0209j,  0.0909+1.0173j, -0.3309+0.4834j,
        -0.3796-1.3598j, -1.1410+0.7581j,  2.0789-1.5162j, -0.5641-0.7436j,
        -0.2081+1.0429j, -0.3351+0.1575j], dtype=torch.complex128)

In [12]:
torch.norm(finufft_out - against_torch) / N

tensor(2.0384e-07, dtype=torch.float64)

In [13]:
Ns = [10, 100, 1000, 10000]

In [20]:
cases = [
    ([1.0, 2.0, 1.0, -1.0, 1.5]),
    # (standard_normal(size=Ns[0]) + 1j * standard_normal(size=Ns[0])),
    # (standard_normal(size=Ns[1]) + 1j * standard_normal(size=Ns[1])),
    # (standard_normal(size=Ns[2]) + 1j * standard_normal(size=Ns[2])),
    # (standard_normal(size=Ns[3]) + 1j * standard_normal(size=Ns[3])),
]

In [21]:
for case in cases:
    N = len(case)
    inv_targets = scipy.fft.fft(case)
    inv_targets_tens = torch.from_numpy(inv_targets)

    against_torch = torch.fft.ifft(inv_targets_tens)
    against_numpy = np.fft.ifft(inv_targets)
    against_scipy = scipy.fft.ifft(inv_targets)

    finufft_out = (
        pytorch_finufft.functional.finufft1D2.forward(
            2 * np.pi * torch.arange(0, 1, 1 / N, dtype=torch.float64),
            inv_target_tens,
            isign=1,
            modeord=1,
        )
        / N
    )

    print(torch.norm(finufft_out - against_torch) / N)

tensor(1.7693, dtype=torch.float64)


In [26]:
targets = cases[0]
N = len(targets)
inv_targets = scipy.fft.fft(case)
inv_target_tens = torch.from_numpy(inv_targets)

against_torch = torch.fft.ifft(inv_targets_tens)
against_numpy = np.fft.ifft(inv_targets)
against_scipy = scipy.fft.ifft(inv_targets)

finufft_out = (
    pytorch_finufft.functional.finufft1D2.forward(
        2 * np.pi * torch.arange(0, 1, 1 / N, dtype=torch.float64),
        inv_target_tens,
        isign=1,
        modeord=1,
    )
    / N
)

print(against_torch)
print(finufft_out)

tensor([ 1.0000+0.j,  2.0000+0.j,  1.0000+0.j, -1.0000+0.j,  1.5000+0.j],
       dtype=torch.complex128)
tensor([ 1.0000+0.j,  2.0000+0.j,  1.0000+0.j, -1.0000+0.j,  1.5000+0.j],
       dtype=torch.complex128)
